# Data Journalism Lesson 6: Mutating data

Learn how to make new columns of data.

In [ ]:
# Setup: Import libraries and define helper functions
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Helper function to check exercise results (similar to previous notebooks)
def check_result(result, expected, check_dtype=False, check_like=True, rtol=1e-3):
    """Check if the result DataFrame matches the expected DataFrame."""
    try:
        # Reset index if necessary to compare values row by row regardless of index
        res = result.reset_index(drop=True)
        exp = expected.reset_index(drop=True)
        pd.testing.assert_frame_equal(res, exp, check_dtype=check_dtype, check_like=check_like, rtol=rtol)
        display(HTML('<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;">' +
                 '<strong>Great work!</strong></div>'))
    except AssertionError as e:
        # print(e) # Uncomment for debugging
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;">' +
                 '<strong>Not quite!</strong> Check your calculations, column names, and sorting.</div>'))

In [ ]:
# Setup: Load data and pre-calculate expected results
import pandas as pd
import numpy as np

# Default state for the exercise (matching previous notebooks)
state_name = "Minnesota"
state_abbr = "MN" # Used for county language lookup

# Load data
try:
    counties = pd.read_csv(f"../_static/census-estimates/{state_name.lower()}.csv")
except FileNotFoundError:
    print(f"Error: Data file for {state_name} not found. Please check the path or state name.")
    # Create an empty DataFrame to avoid errors later if file not found
    counties = pd.DataFrame()

# Define county language based on state (Python equivalent of R's case_when)
county_language_map = {
    "AK": ("county equivalent", "county equivalents"),
    "CT": ("planning region", "planning regions"),
    "LA": ("parish", "parishes"),
    "VA": ("county or city", "counties or cities")
}
county_language = county_language_map.get(state_abbr, ("county", "counties"))

# Pre-calculate expected results for exercises
if not counties.empty:
    nrows = counties.shape[0]
    ncols = counties.shape[1]

    # Exercise 1 & 2 expected (mutate)
    counties_with_pct_change = counties.copy() # Avoid modifying original
    counties_with_pct_change['percent_change'] = ((counties_with_pct_change['POPESTIMATE2023'] - counties_with_pct_change['POPESTIMATE2022']) / counties_with_pct_change['POPESTIMATE2022']) * 100

    # Exercise 3 expected (arrange)
    counties_sorted_by_pct_change = counties_with_pct_change.sort_values('percent_change', ascending=False)
    fastest_county_expected = counties_sorted_by_pct_change.head(1)

    # Exercise 4 expected (raw change and arrange)
    counties_with_raw_change = counties.copy()
    counties_with_raw_change['rawchange'] = counties_with_raw_change['POPESTIMATE2023'] - counties_with_raw_change['POPESTIMATE2022']
    counties_sorted_by_raw_change = counties_with_raw_change.sort_values('rawchange', ascending=False)
    biggest_county_expected = counties_sorted_by_raw_change.head(1)

    # Exercise 5 expected (saving the result)
    countieswithrawchange_expected = counties_sorted_by_raw_change
else:
    nrows, ncols = 0, 0
    fastest_county_expected = pd.DataFrame()
    biggest_county_expected = pd.DataFrame()
    countieswithrawchange_expected = pd.DataFrame()

In [ ]:
# Setup: Glue variables for use in markdown
from myst_nb import glue

if not counties.empty:
    glue("state_name", state_name, display=False)
    glue("county_singular", county_language[0], display=False)
    glue("county_plural", county_language[1], display=False)
    glue("fastest_county_name", fastest_county_expected['CTYNAME'].iloc[0] if not fastest_county_expected.empty else "N/A", display=False)
    glue("fastest_county_state", fastest_county_expected['STNAME'].iloc[0] if not fastest_county_expected.empty else "N/A", display=False)
    glue("fastest_county_pct", f"{fastest_county_expected['percent_change'].iloc[0]:.1f}" if not fastest_county_expected.empty else "N/A", display=False)
    glue("biggest_county_name", biggest_county_expected['CTYNAME'].iloc[0] if not biggest_county_expected.empty else "N/A", display=False)
    glue("biggest_county_state", biggest_county_expected['STNAME'].iloc[0] if not biggest_county_expected.empty else "N/A", display=False)
    glue("biggest_county_raw", f"{biggest_county_expected['rawchange'].iloc[0]:,}" if not biggest_county_expected.empty else "N/A", display=False)
    glue("num_rows", f"{nrows:,}", display=False)
    glue("num_cols", f"{ncols:,}", display=False)
else:
    # Glue placeholders if data loading failed
    glue("state_name", state_name, display=False)
    glue("county_singular", "county", display=False)
    glue("county_plural", "counties", display=False)
    glue("fastest_county_name", "N/A", display=False)
    glue("fastest_county_state", "N/A", display=False)
    glue("fastest_county_pct", "N/A", display=False)
    glue("biggest_county_name", "N/A", display=False)
    glue("biggest_county_state", "N/A", display=False)
    glue("biggest_county_raw", "N/A", display=False)
    glue("num_rows", "0", display=False)
    glue("num_cols", "0", display=False)

## The Goal

In this lesson, you'll learn about mutating data - a powerful technique for creating new columns based on existing data. By the end of this tutorial, you'll understand how to use pandas to perform calculations, such as percent change, on your data by assigning results to new columns. You'll also learn how to combine this with other pandas functions like `.sort_values()` to answer specific questions about your dataset. This skill is essential for data journalists who often need to derive new insights from raw data.

## What is Data Journalism?

One of the first lessons I learned about data journalism came from Sarah Cohen, Pulitzer Prize-winning data editor at the Washington Post and New York Times now Knight Chair of Data Journalism at Arizona State University. She said that numbers, in isolation, are often meaningless. One of the key guiding questions that drives data journalism is simply this:

"As compared to what?"

You have a number. Great. What was it last month? Or last year? Or 5/10/20 years ago? Was it up or down?

If your number is for a geography, how does your geography compare to others?

How does it compare to the average? Nationally or statewide?

For a long, long time, this question has sat square in my head as I'm analyzing data. As compared to what? Today, we start building skills to answer that question.

## The Basics

One of the most common data analysis techniques is to look at change over time. The most common way of comparing change over time is through percent change. The math behind calculating percent change is very simple, and you should know it off the top of your head. The easy way to remember it is:

`(new - old) / old`

Or new minus old divided by old. Your new number minus the old number, the result of which is divided by the old number. To do that in Python with pandas, we can calculate new metrics in a new column using existing columns of data.

So first we'll import pandas so we can read in our data and begin to work with it.

In [ ]:
import pandas as pd

Now we'll import a common and simple dataset of county population estimates from the U.S. Census Bureau. Each year, the Census Bureau publishes estimates for states and counties. This one has every {glue:text}`county_singular` in {glue:text}`state_name`. A common question: who are the winners and losers?

Your first task is to import the data. The code to do this was run in the setup cell, loading data for {glue:text}`state_name` into the `counties` DataFrame.

In [ ]:
counties = pd.read_csv("../_static/census-estimates/minnesota.csv")

Remember, if you want to see the first few rows -- handy to take a peek at your data -- you can use the `.head()` method.

In [ ]:
counties.____()

### Exercise 1: Calculating percent change

The code to calculate percent change is pretty simple. In pandas, we create a new column by assigning the result of a calculation to it using square brackets `[]` and the new column name.

We're trying to do `(new - old) / old`, but we're doing it with column names.

If we look at what we got when we imported the data, you'll see there's a column called `POPESTIMATE2023` as the new data, and we'll use `POPESTIMATE2022` as the old data. So we're looking at one year's change.

Create a new column called `percent_change` in the `counties` DataFrame. Calculate the value using the formula `(POPESTIMATE2023 - POPESTIMATE2022) / POPESTIMATE2022`.

In [ ]:
# Make a copy to avoid modifying the original DataFrame in this step
counties_calc = counties.copy()

counties_calc['_____'] = (counties_calc[_____] - counties_calc[_____]) / counties_calc[_____]

# Display the first few rows with the new column
print(counties_calc.head())

# Check the calculation (ignoring the multiplication by 100 for now)
check_result(counties_calc[['percent_change']], counties_with_pct_change[['percent_change']] / 100, rtol=1e-5)

Look out on the right and you'll see your `percent_change` column. But what do you see right away? Do those numbers look like we expect them to? No.


### Exercise 2: Converting to a percentage

Those numbers are a decimal -- a really, really small decimal when you see things like e-03, which means that number is multiplied by 10<sup>-3</sup> (move the decimal three spaces to the left). We want it expressed as a percentage. So let's fix that by multiplying by 100.

Modify the calculation from Exercise 1 to multiply the result by 100.

In [ ]:
# Make a copy again
counties_calc_pct = counties.copy()

counties_calc_pct['percent_change'] = ((counties_calc_pct[_____] - counties_calc_pct[_____]) / counties_calc_pct[_____]) * _____

# Display the first few rows
print(counties_calc_pct.head())

# Check the result
check_result(counties_calc_pct[['percent_change']], counties_with_pct_change[['percent_change']])

Better.

But what order is that in? Note: not by percent change. It's alphabetical by {glue:text}`county_singular`. So you've answered the question of which {glue:text}`county_singular` appears first alphabetically in {glue:text}`state_name`. News? Not hardly.

### Exercise 3: Arranging to answer a question

So who had the most growth in that one year? We can find that with `.sort_values()`, which we learned about previously.

Take the `counties_calc_pct` DataFrame (which now includes the correct `percent_change`), and sort it by the `percent_change` column in descending order to find the fastest-growing {glue:text}`county_plural`.

In [ ]:
# Start with the DataFrame that has the percent_change column
counties_sorted = counties_calc_pct.sort_values(by=_____, ascending=_____)

# Display the top rows
print(counties_sorted.head())

# Check the result
check_result(counties_sorted, counties_sorted_by_pct_change)

Which {glue:text}`county_singular` is the fastest growing? {glue:text}`fastest_county_name`, {glue:text}`fastest_county_state`, which grew {glue:text}`fastest_county_pct`% in just one year.

## More mutating

### Exercise 4: Looking at change differently

With this data, we have every {glue:text}`county_singular` and the population in two years. We've looked at the percent change, which allows us to put Los Angeles County, California -- the most populous in the nation -- and Loving County, Texas -- the least populous -- on the same playing field. But what if we wanted to know which {glue:text}`county_singular` was growing just by the raw number of people?

What you're going to learn about data journalism is that often, the simplest way is going to be the best. Simple math means you can explain what you've done easily to anyone. So how do we get a raw number? Subtraction!

Create a new column called `rawchange` in the `counties` DataFrame by subtracting `POPESTIMATE2022` from `POPESTIMATE2023`. Then, sort the DataFrame by this new `rawchange` column in descending order.

In [ ]:
# Start with the original counties DataFrame and make a copy
counties_raw = counties.copy()

# Calculate raw change
counties_raw[_____] = counties_raw[_____] - counties_raw[_____]

# Sort by raw change
counties_raw_sorted = counties_raw.sort_values(by=_____, ascending=_____)

# Display the top rows
print(counties_raw_sorted.head())

# Check the result
check_result(counties_raw_sorted, counties_sorted_by_raw_change)

What {glue:text}`county_singular` added the most people by raw number? It's {glue:text}`biggest_county_name`, {glue:text}`biggest_county_state`} adding {glue:text}`biggest_county_raw` people in one year. Know where that is? What might explain that growth? Those are reporter questions that make stories.

Now, we've spent a fair amount of time looking at creating new columns, but we haven't gone over a fundamental part of working with data in a programming language. Quite simply, it's this: **Unless you explicitly tell your code to save or modify something permanently, the change might only exist temporarily.** Don't believe me? You created `percent_change` and `rawchange` in the exercises above, right? So you should see them in the original `counties` DataFrame, right?

Take a look using `.info()` which gives a summary including column names and types.

In [ ]:
counties._____

Your new columns always get added to the end of your DataFrame, so look at the list of columns.

Where are your columns? The truth is, they existed only in the temporary DataFrames (`counties_calc`, `counties_calc_pct`, `counties_raw`, `counties_raw_sorted`) we created in each step. We never told Python to permanently add them to the *original* `counties` DataFrame or to save those temporary DataFrames under a specific name for later use.

A good rule of thumb: If you perform an operation and want to use the result later, you need to assign it to a variable (either overwriting the original or creating a new one). In pandas, many operations return a *new* DataFrame by default, rather than modifying the original one in place.

```{admonition} Key Concept
:class: info
When you perform calculations or manipulations (like adding a column or sorting), the result is often a *new* DataFrame. To keep these changes, you must assign the result to a variable using `=`. If you assign back to the original variable name (`df = df.sort_values(...)`), you overwrite the original. If you assign to a new name (`df_sorted = df.sort_values(...)`), you keep both.
```

So how do you save your work? You've been doing it all along when loading data:
`counties = pd.read_csv(...)`
The `=` sign assigns the result of `pd.read_csv(...)` to the variable `counties`.

### Exercise 5: Save to a new DataFrame

The `=` just takes what is on the right side and saves it into the variable name on the left side. So you could run `my_variable = "Some text"` and it will run without error. You just saved the text into a variable.

All this means is that you can do the same thing with the results of your code. Let's save the code we wrote to calculate `rawchange` and sort by it (from Exercise 4) into a *new* DataFrame called `countieswithrawchange`.

Assign the result of calculating `rawchange` and sorting the `counties` DataFrame to the new variable `countieswithrawchange`.

In [ ]:
_____ = counties.copy() # Start with a copy
_____[_____] = _____[_____] - _____[_____]
_____ = _____.sort_values(by=_____, ascending=_____)

# No output is printed here because we assigned the result to a variable
# print(countieswithrawchange.head()) # You could uncomment this to see it

Some of you, right now, might be wondering: Where is my data? Why don't I see it printed below the cell? And that means you either didn't read above or you don't remember it only a few paragraphs later. When you use `=`, that means save it (assign it to the variable). It does not automatically mean print it or display it. So your code is doing what you told it to do: save the results to a new DataFrame called `countieswithrawchange`.

Why did I give it a new name and not just use `counties`? A good habit to get into is often *not* to overwrite your original data, especially during exploration. Make new DataFrames each time you make a significant change. Why? Because you might need to go back to the original DataFrame or an intermediate step to find where you went wrong. And if you're overwriting your original data, that becomes much, much harder. A useful pattern: Change data? Save to a new DataFrame variable. Use the new DataFrame. Repeat as needed.

You can see that it worked by using `.info()` on your new DataFrame (or just printing its `.head()`).

In [ ]:
countieswithrawchange._____

See that last column name? There's `rawchange`. That means if you want to use it in the future, you have to use the `countieswithrawchange` DataFrame instead of the original `counties` DataFrame.

## The Recap

Throughout this lesson, you've learned how to create new columns in your dataset using pandas by assigning the results of calculations. You've calculated percent change and raw change in population data, demonstrating how to perform simple mathematical operations on existing columns. You've also combined creating new columns with `.sort_values()` to identify the fastest-growing {glue:text}`county_plural` and those with the largest absolute population increases. Remember, creating new columns (mutating data) allows you to uncover new insights and tell more complex stories with your data.

## Terms to Know

- **Mutating**: Creating new columns or modifying existing ones in a DataFrame, often by assigning the result of a calculation.
- **Column Assignment**: Using square brackets `[]` with a new column name to add or update a column in a pandas DataFrame (e.g., `df['new_col'] = ...`).
- **`.assign()`**: A pandas method that returns a *new* DataFrame with new columns added or existing columns assigned new values.
- **Percent change**: A calculation showing the relative change between two values (`(new - old) / old * 100`).
- **Raw change**: The absolute difference between two values (`new - old`).
- **Method Chaining**: Linking multiple pandas operations together (e.g., `df.assign(...).sort_values(...)`).
- **`.sort_values()`**: A pandas method used to sort a DataFrame based on one or more columns.
- **`ascending=False`**: An argument used within `.sort_values()` to sort data in descending order.